这里说频率是指一个机场航班出现的频率

## 加载数据

数据已经预处理过了

In [1]:
import pandas as pd

# 加载数据
data = pd.read_csv('./pre_2023-2025_with_comp_train.csv', dtype={'flt_no': str})

# 查看前几行数据，确保加载成功
# 显示前两行数据以确保正确加载
print(data.shape)
print(data.head(5))
print(data.tail(5))

(1507927, 20)
  flt_no    cap aircraft  legs  leg_no  duration  pax    a    b    c  \
0   7558  110.0      195     1       1      1.30   97  AAT  URC  NaN   
1   7470  110.0      195     1       1      1.25   67  ACF  URC  NaN   
2   769R   93.0      190     3       3      4.75   22  ACF  TLQ  XIY   
3   5248  162.0      320     1       1      1.83   55  AKA  HGH  NaN   
4   6250  167.0      320     1       1      3.98  166  AKU  CGO  NaN   

    unit_price  competitor_price  year  month  day  weekday  hour  minute  \
0   470.474227        -94.900773  2023      1    1        6    14      35   
1   454.925373       -123.872095  2023      1    1        6    22      40   
2  1177.818182          0.000000  2023      1    1        6    18       0   
3   669.090909          0.000000  2023      1    1        6    12      55   
4  1794.783133        -65.028989  2023      1    1        6    13      10   

  from   to  
0  AAT  URC  
1  ACF  URC  
2  ACF  XIY  
3  AKA  HGH  
4  AKU  CGO  
      

In [2]:
# 检查标准化后的统计信息
print("\n标准化后的统计信息：")
print(data['pax'].describe())


标准化后的统计信息：
count    1.507927e+06
mean     1.213228e+02
std      5.255228e+01
min      2.000000e+01
25%      8.100000e+01
50%      1.260000e+02
75%      1.590000e+02
max      3.110000e+02
Name: pax, dtype: float64


## 编码分类变量

### 新增城市标签

In [3]:
import json
# 加载字典
with open('../../my/encoder/city_labels_航班频率加权图标签.json', 'r') as file:
    city_labels_loaded = json.load(file)

print("加载的字典：", city_labels_loaded)

加载的字典： {'AAT': 1, 'CKG': 2, 'PEK': 0, 'TCG': 1, 'URC': 2, 'XIY': 2, 'YIN': 1, 'ACF': 1, 'HMI': 2, 'SHF': 1, 'TLQ': 1, 'ACX': 1, 'CGO': 2, 'KMG': 2, 'AEB': 1, 'CAN': 2, 'HGH': 2, 'KWL': 2, 'TFU': 2, 'AKA': 1, 'CSX': 2, 'HAK': 2, 'TSN': 2, 'UYN': 2, 'AKU': 1, 'FOC': 2, 'HTN': 1, 'AOG': 2, 'TNA': 2, 'AQG': 2, 'KWE': 2, 'NGB': 2, 'TAO': 2, 'XMN': 2, 'AVA': 2, 'LLB': 2, 'BAR': 0, 'HRB': 2, 'JMJ': 1, 'KHN': 2, 'LHW': 2, 'NNG': 2, 'SJW': 2, 'SZX': 2, 'BAV': 2, 'HLD': 2, 'TGO': 2, 'XIL': 2, 'BFJ': 2, 'BHY': 2, 'HET': 2, 'HFE': 2, 'HNY': 2, 'JNG': 2, 'NAO': 1, 'WUH': 2, 'XUZ': 2, 'YIH': 2, 'BPE': 1, 'BPL': 1, 'BPX': 1, 'BSD': 1, 'BZX': 2, 'JJN': 2, 'LJG': 2, 'NKG': 2, 'CGQ': 2, 'DLC': 2, 'DOY': 0, 'HDG': 2, 'HSN': 2, 'HZG': 1, 'INC': 2, 'JIQ': 0, 'PKX': 2, 'PVG': 2, 'SHA': 0, 'SHE': 2, 'SQJ': 2, 'SYX': 2, 'TEN': 2, 'TYN': 2, 'WEF': 2, 'WUA': 2, 'YIC': 2, 'ZQZ': 2, 'CGD': 2, 'JHG': 2, 'CIF': 2, 'DLU': 1, 'DXJ': 1, 'ENH': 2, 'GOQ': 1, 'HUZ': 2, 'JGS': 1, 'JNZ': 1, 'KCA': 1, 'KHG': 1, 'KOW': 2, 'K

In [4]:
# 使用 map 对 'a', 'b', 'c', 'from', 'to' 列进行标签化，新增对应的标签列
data['a_label'] = data['a'].map(city_labels_loaded)
data['b_label'] = data['b'].map(city_labels_loaded)
data['c_label'] = data['c'].map(city_labels_loaded)
data['from_label'] = data['from'].map(city_labels_loaded)
data['to_label'] = data['to'].map(city_labels_loaded)

### 新增城市二维嵌入

In [5]:
import json

# 加载字典
with open('../../my/encoder/城市嵌入编码_航班频率加权图.json', 'r') as file:
    city_embeddings = json.load(file)

print("加载的字典：", city_embeddings)


加载的字典： {'AAT': [-1.37530517578125, -0.35888153314590454], 'CKG': [0.01064071711152792, -0.8891868591308594], 'PEK': [0.19175417721271515, -0.1063457801938057], 'TCG': [-1.4468026161193848, -0.36743783950805664], 'URC': [-1.2777715921401978, -0.6882389783859253], 'XIY': [0.4810418486595154, -0.8047342300415039], 'YIN': [-1.9216420650482178, -0.1084386482834816], 'ACF': [-1.6529606580734253, -0.20273223519325256], 'HMI': [-0.609366774559021, -0.7292864918708801], 'SHF': [-1.001150131225586, 0.16054262220859528], 'TLQ': [-0.8284773826599121, 0.0521150678396225], 'ACX': [-0.6958959102630615, 0.2985990047454834], 'CGO': [-0.32369911670684814, -0.8481274843215942], 'KMG': [0.3200676739215851, -0.3121274411678314], 'AEB': [-0.5060219168663025, 0.7377899885177612], 'CAN': [0.04243234544992447, -0.4815085530281067], 'HGH': [0.02870369330048561, -0.4355216324329376], 'KWL': [0.6726647615432739, -0.7483685612678528], 'TFU': [-0.16380424797534943, -0.40230828523635864], 'AKA': [-0.2005784809589386

In [6]:
import pandas as pd
import numpy as np

# 将 city_embeddings 转换为 DataFrame
embedding_df = pd.DataFrame.from_dict(city_embeddings, orient='index', columns=['embedding_1', 'embedding_2'])
embedding_df.index.name = 'city'

# 用 'a', 'b', 'c', 'from', 'to' 字段与 embedding_df 合并
data = data.merge(embedding_df, left_on='a', right_index=True, how='left')
data.rename(columns={'embedding_1': 'a_embedding_1', 'embedding_2': 'a_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='b', right_index=True, how='left')
data.rename(columns={'embedding_1': 'b_embedding_1', 'embedding_2': 'b_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='c', right_index=True, how='left')
data.rename(columns={'embedding_1': 'c_embedding_1', 'embedding_2': 'c_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='from', right_index=True, how='left')
data.rename(columns={'embedding_1': 'from_embedding_1', 'embedding_2': 'from_embedding_2'}, inplace=True)

data = data.merge(embedding_df, left_on='to', right_index=True, how='left')
data.rename(columns={'embedding_1': 'to_embedding_1', 'embedding_2': 'to_embedding_2'}, inplace=True)

# 查看添加的新列
print(data[['a_embedding_1', 'a_embedding_2', 'b_embedding_1', 'b_embedding_2', 'c_embedding_1', 'c_embedding_2', 'from_embedding_1', 'from_embedding_2', 'to_embedding_1', 'to_embedding_2']])


         a_embedding_1  a_embedding_2  b_embedding_1  b_embedding_2  \
0            -1.375305      -0.358882      -1.277772      -0.688239   
1            -1.652961      -0.202732      -1.277772      -0.688239   
2            -1.652961      -0.202732      -0.828477       0.052115   
3            -0.200578       0.401379       0.028704      -0.435522   
4            -1.645835      -0.466154      -0.323699      -0.848127   
...                ...            ...            ...            ...   
1507922       0.264739      -0.933443       0.481042      -0.804734   
1507923       0.264739      -0.933443       0.481042      -0.804734   
1507924       0.264739      -0.933443       0.481042      -0.804734   
1507925       0.481042      -0.804734       0.491313      -0.729582   
1507926       0.359681      -0.616342       0.491313      -0.729582   

         c_embedding_1  c_embedding_2  from_embedding_1  from_embedding_2  \
0                  NaN            NaN         -1.375305         -0.358

### 频率编码

使用 json 保存和加载 city_map

In [7]:
# 加载city_map
with open('../../my/encoder/city_map_频率编码.json', 'r') as f:
    city_map = json.load(f)

# 使用 city_map 替换指定列的值
columns_to_replace = ['a', 'b', 'c', 'from', 'to']

# 遍历指定列并直接用 map 映射
for col in columns_to_replace:
    data[col] = data[col].map(city_map)


print(data)

        flt_no    cap aircraft  legs  leg_no  duration  pax       a       b  \
0         7558  110.0      195     1       1      1.30   97    2333  137647   
1         7470  110.0      195     1       1      1.25   67    2434  137647   
2         769R   93.0      190     3       3      4.75   22    2434     810   
3         5248  162.0      320     1       1      1.83   55    1401  106773   
4         6250  167.0      320     1       1      3.98  166    6066  135409   
...        ...    ...      ...   ...     ...       ...  ...     ...     ...   
1507922   7690  154.0      32F     3       3      6.53   34   33327  192653   
1507923   7690  154.0      32F     3       1      2.78  113   33327  192653   
1507924   8304  167.0      738     1       1      2.48  158   33327  192653   
1507925   7545  155.0      32F     3       2      2.05   74  192653    5100   
1507926   7546  155.0      32F     3       2      1.42  136  128149    5100   

                c  ...  a_embedding_1  a_embedding_

### 'flt_no', 'bd_type', 'aircraft'编码

In [8]:
import joblib
from sklearn.preprocessing import LabelEncoder
import os

# 定义需要编码的分类特征
# categorical_columns = ['flt_no', 'bd_type', 'aircraft']
categorical_columns = ['flt_no', 'aircraft']

# 从保存的文件中加载编码器并应用到data
for col in categorical_columns:
    # 加载编码器
    encoder_path = os.path.join('../../my/encoder/', f"{col}_encoder_all.pkl")
    le = joblib.load(encoder_path)
    
    try:
        # 对data进行转换
        data[col] = le.transform(data[col])
        print(f"{col}列编码完成")
    except ValueError as e:
        # 如果遇到新的类别，打印错误信息
        print(f"{col}列编码出错: {str(e)}")
        # 找出新的类别
        new_categories = set(data[col]) - set(le.classes_)
        print(f"{col}列中的新类别: {new_categories}")

# 查看编码后的结果
print("\n编码后的前几行数据：")
print(data[categorical_columns].head())

flt_no列编码完成
aircraft列编码完成

编码后的前几行数据：
   flt_no  aircraft
0    2945         2
1    2821         2
2    3159         0
3     661         9
4    1507         9


### 新增pax比例字段

## 特征和目标分离
我们要预测的是pax字段，其他字段作为特征。

In [9]:
# 特征列
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft',  'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','holiday', 'hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price']]
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label']]

# 有abc，有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
# X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]
X = data[['flt_no', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'a', 'b', 'c', 'year', 'month', 'day', 'weekday','hour', 'minute', 'from', 'to','unit_price','competitor_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 删除了abc，但有标签，有嵌入
# X = data[['flt_no', 'bd_type', 'cap', 'aircraft', 'legs', 'leg_no', 'duration', 'year', 'month', 'day', 'weekday','hour', 'minute', 'second', 'unit_price','a_label' ,'b_label' ,'c_label' ,'from_label' ,'to_label','a_embedding_1' , 'a_embedding_2' , 'b_embedding_1','b_embedding_2' , 'c_embedding_1' , 'c_embedding_2' , 'from_embedding_1','from_embedding_2' , 'to_embedding_1' , 'to_embedding_2']]

# 目标列
y = data['pax']

### 对x进行标准化

In [10]:
from sklearn.preprocessing import StandardScaler

# 对所有特征进行标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 将标准化后的数据转回 DataFrame 格式
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

# 查看结果
pd.set_option('display.max_columns', None)  # 显示所有列
print(X_scaled.head(1))


# 保存 scaler
joblib.dump(scaler, '../../my/encoder/standard_scaler_x.pkl')
print("x的标准化器已保存为 standard_scaler_x.pkl")

     flt_no       cap  aircraft      legs    leg_no  duration         a  \
0  0.451505 -1.477052 -1.324136 -0.865334 -0.565872 -1.227121 -1.436836   

          b   c      year     month       day   weekday     hour    minute  \
0  1.079574 NaN -0.882038 -1.314987 -1.681533  1.483702 -0.00125  0.477048   

       from       to  unit_price  competitor_price   a_label   b_label  \
0 -1.334675  0.82066   -0.552425         -0.500111 -2.017569  0.299796   

   c_label  from_label  to_label  a_embedding_1  a_embedding_2  b_embedding_1  \
0      NaN   -1.982899   0.28426      -2.794979       0.362764       -2.95506   

   b_embedding_2  c_embedding_1  c_embedding_2  from_embedding_1  \
0      -0.423661            NaN            NaN         -2.925336   

   from_embedding_2  to_embedding_1  to_embedding_2  
0          0.350482       -2.736296       -0.385379  
x的标准化器已保存为 standard_scaler_x.pkl


### 对y进行标准化

In [11]:
# 对目标列 y 进行标准化
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y.values.reshape(-1, 1))  # 将 y 转换为 2D 数组进行标准化

# 转换回 DataFrame 格式
y_scaled = pd.DataFrame(y_scaled, columns=['pax_scaled'])

# 查看标准化后的 y
print(y_scaled.head())

# 保存 y 的 scaler
joblib.dump(scaler_y, '../../my/encoder/standard_scaler_y.pkl')
print("y的标准化器已保存为 standard_scaler_y.pkl")

   pax_scaled
0   -0.462832
1   -1.033692
2   -1.889982
3   -1.262036
4    0.850147
y的标准化器已保存为 standard_scaler_y.pkl


## 训练XGBoost模型

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# 自定义 SMAPE 函数
def smape(y_true, y_pred):
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_true - y_pred)
    return np.mean(diff / denominator) * 100

# 自定义评估函数
def smape_eval(y_pred, dtrain):
    y_true = dtrain.get_label()
    smape_value = smape(y_true, y_pred)
    return 'SMAPE', smape_value

# 数据划分
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# 输出数据集大小
print(f'训练集大小: {X_train.shape[0]}')
print(f'验证集大小: {X_val.shape[0]}')
print(f'测试集大小: {X_test.shape[0]}')

# 转换为 DMatrix 格式
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# 设置参数
params = {
    'objective': 'reg:squarederror',
    'learning_rate': 0.1,
    'max_depth': 10,#10效果最好
    'subsample': 0.8,#0.8效果最好
    'colsample_bytree':1,
    'alpha': 1,
    "lambda":1
}

# 设置评估集
evals = [(dtrain, 'train'), (dval, 'validation')]

# 训练模型，使用自定义评估指标
model = xgb.train(
    params,
    dtrain,
    num_boost_round=750,
    evals=evals,
    early_stopping_rounds=10,
    custom_metric=smape_eval,  # 使用 custom_metric 参数
    verbose_eval=10 #隔多少轮显示一次
)

# 预测测试集
y_pred = model.predict(dtest)

# # 测试集 SMAPE 评估
# test_smape = smape(y_test, y_pred)
# print(f'SMAPE on Test Set: {test_smape:.2f}%')

# # 测试集 MSE 评估
# mse = mean_squared_error(y_test, y_pred)
# print(f'Mean Squared Error on Test Set: {mse}')

训练集大小: 1206341
验证集大小: 150793
测试集大小: 150793
[0]	train-rmse:0.92403	train-SMAPE:167.46267	validation-rmse:0.92321	validation-SMAPE:167.48449
[10]	train-rmse:0.53773	train-SMAPE:84.68614	validation-rmse:0.54050	validation-SMAPE:85.13163
[20]	train-rmse:0.44796	train-SMAPE:71.28609	validation-rmse:0.45395	validation-SMAPE:71.95600
[30]	train-rmse:0.42080	train-SMAPE:66.91840	validation-rmse:0.42909	validation-SMAPE:67.80453
[40]	train-rmse:0.40740	train-SMAPE:64.85798	validation-rmse:0.41744	validation-SMAPE:65.95458
[50]	train-rmse:0.39784	train-SMAPE:63.49273	validation-rmse:0.40939	validation-SMAPE:64.72021
[60]	train-rmse:0.39010	train-SMAPE:62.43958	validation-rmse:0.40292	validation-SMAPE:63.78160
[70]	train-rmse:0.38491	train-SMAPE:61.75492	validation-rmse:0.39907	validation-SMAPE:63.21281
[80]	train-rmse:0.38056	train-SMAPE:61.20775	validation-rmse:0.39571	validation-SMAPE:62.75727
[90]	train-rmse:0.37649	train-SMAPE:60.69457	validation-rmse:0.39259	validation-SMAPE:62.35096
[100]	

In [13]:
y_pred

array([-0.17882934,  0.6666585 , -0.45674452, ...,  0.9540744 ,
        2.4976468 ,  0.5420883 ], dtype=float32)

In [14]:
# 反标准化 y_test
y_test_original = scaler_y.inverse_transform(y_test.values.reshape(-1, 1))

# 反标准化预测结果 y_pred
y_pred_original = scaler_y.inverse_transform(y_pred.reshape(-1, 1))

In [15]:
y_test_original

array([[ 95.],
       [162.],
       [ 51.],
       ...,
       [180.],
       [268.],
       [139.]])

In [16]:
y_pred_original

array([[111.92496],
       [156.35725],
       [ 97.31989],
       ...,
       [171.46161],
       [252.57983],
       [149.8108 ]], dtype=float32)

In [17]:
# 正确的写法
test_results = list(zip(y_test_original[:100], y_pred_original[:100]))  # 真实值和预测值

print("\n20条测试结果（真实值 vs 预测值）:")
for i, (true_value, pred_value) in enumerate(test_results[:40]):
    # 如果是多维数组，使用 .item() 转换为标量
    true_value = true_value.item() if isinstance(true_value, np.ndarray) else true_value
    pred_value = pred_value.item() if isinstance(pred_value, np.ndarray) else pred_value
    print(f"第{i+1}条: 真实值={true_value}, 预测值={pred_value:.2f}")



20条测试结果（真实值 vs 预测值）:
第1条: 真实值=95.0, 预测值=111.92
第2条: 真实值=162.0, 预测值=156.36
第3条: 真实值=51.0, 预测值=97.32
第4条: 真实值=69.0, 预测值=76.69
第5条: 真实值=49.0, 预测值=62.56
第6条: 真实值=159.0, 预测值=161.34
第7条: 真实值=26.0, 预测值=30.83
第8条: 真实值=143.0, 预测值=145.40
第9条: 真实值=104.0, 预测值=115.09
第10条: 真实值=150.0, 预测值=138.01
第11条: 真实值=161.0, 预测值=143.57
第12条: 真实值=146.0, 预测值=121.69
第13条: 真实值=101.0, 预测值=151.01
第14条: 真实值=117.0, 预测值=100.97
第15条: 真实值=123.0, 预测值=144.79
第16条: 真实值=126.0, 预测值=138.97
第17条: 真实值=73.0, 预测值=64.66
第18条: 真实值=97.0, 预测值=137.16
第19条: 真实值=40.0, 预测值=42.68
第20条: 真实值=153.0, 预测值=145.50
第21条: 真实值=107.0, 预测值=126.13
第22条: 真实值=26.0, 预测值=52.28
第23条: 真实值=242.0, 预测值=245.85
第24条: 真实值=48.0, 预测值=50.85
第25条: 真实值=173.0, 预测值=170.76
第26条: 真实值=145.0, 预测值=136.00
第27条: 真实值=83.0, 预测值=85.60
第28条: 真实值=174.0, 预测值=155.58
第29条: 真实值=93.0, 预测值=84.40
第30条: 真实值=67.0, 预测值=75.51
第31条: 真实值=177.0, 预测值=172.91
第32条: 真实值=68.0, 预测值=77.68
第33条: 真实值=39.0, 预测值=45.04
第34条: 真实值=125.0, 预测值=115.44
第35条: 真实值=98.0, 预测值=86.05
第36条: 真实值=156.0, 预测值=149.01
第37条: 真实值

In [18]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

def calculate_smape(y_true, y_pred):
    """
    计算 Symmetric Mean Absolute Percentage Error (SMAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0
    diff = np.abs(y_pred - y_true)
    
    # 避免除以零，将分母中为零的项替换为一个小值
    denominator = np.where(denominator == 0, 1e-8, denominator)
    
    smape = 100 * np.mean(diff / denominator)
    return smape


def calculate_mape(y_true, y_pred):
    """
    计算 Mean Absolute Percentage Error (MAPE)
    """
    y_true, y_pred = np.array(y_true).ravel(), np.array(y_pred).ravel()
    
    # 避免除以零，将 y_true 中的零值替换为一个小值
    y_true = np.where(y_true == 0, 1e-8, y_true)
    
    mape = 100 * np.mean(np.abs((y_true - y_pred) / y_true))
    return mape


# 假设 y_test 和 y_pred 已经是标准化反归一化后的数据
# 将其转换为一维数组以确保形状一致
y_test = np.array(y_test_original).ravel()
y_pred = np.array(y_pred_original).ravel()

# 评估指标
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
mape = calculate_mape(y_test, y_pred)
smape = calculate_smape(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 打印结果
print(f'Mean Squared Error (MSE): {mse:.4f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.4f}')
print(f'Mean Absolute Error (MAE): {mae:.4f}')
print(f'Mean Absolute Percentage Error (MAPE): {mape:.4f}%')
print(f'Symmetric Mean Absolute Percentage Error (SMAPE): {smape:.4f}%')
print(f'R-squared (R²): {r2:.4f}')

Mean Squared Error (MSE): 323.7294
Root Mean Squared Error (RMSE): 17.9925
Mean Absolute Error (MAE): 13.4850
Mean Absolute Percentage Error (MAPE): 15.5243%
Symmetric Mean Absolute Percentage Error (SMAPE): 14.0045%
R-squared (R²): 0.8829


似乎对于较小值预测存在误差

## 保存模型

In [19]:
model.save_model("../../my/model/频率编码/归一化_xgboost_model_1000.json")
print("模型已保存为 xgboost_model_1000.json")

模型已保存为 xgboost_model_1000.json


## 超参数设置

## 不同特征重要程度测试

In [20]:
# import xgboost as xgb
# import matplotlib.pyplot as plt

# # 假设 model 是训练好的 XGBoost 模型
# xgb.plot_importance(model, importance_type='weight', title="Feature Importance (Weight)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='gain', title="Feature Importance (Gain)", height=0.5)
# plt.show()

# xgb.plot_importance(model, importance_type='cover', title="Feature Importance (Cover)", height=0.5)
# plt.show()